<font color='red'>**Note:** non-HBP members should contact “support@humanbrinproject.eu” for access to the validation tools</font>

## About this test
This test shall take as input a directory containing neuronal morphologies (currently supports `swc` format). The user decides whether to run the validations for all available morphologies, or a subset of these.

The validations are carried out using 'NeuroM' (https://github.com/BlueBrain/NeuroM). The validation test evaluates the morphology in two stages:

1. **Hard Constraints**

   Here we evaluate the integrity of the neuronal reconstruction in order to determine if it is appropriate for further evaluations. The evaluations here can be sub-divided into the following NeuroM features (apps):
   
   - morph_check
   - cut_plane_detection <br /><br />
   
2. **Soft Constraints** [Currently only available for *Fast Spiking Interneurons*]

     Neuronal reconstructions that pass the 'Hard Constraints' are evaluated here for their morphometric features. The features are extracted using NeuroM's morph_stats app, wherever possible, either directly or as a combination of multiple features. These are then compared against experimentally obtained data, as determined by the particular validation test being executed.
     
     Some of the features currently included are soma's diameter and the maximal branch order in the dendrites, besides the number of trunk sections, -X,Y,Z- extents, field's diameter and total path-length of both the axon and the dendrites.
     
     By the present, only FS interneurons can be considered, since observation data is missing for other neuron types.

### Installing the required packages
(no user intervention required)

In [ ]:
import os
import pkg_resources
from pkg_resources import parse_version
!pip install -q tornado==4.5.3

<font color='red'>**Note:** If you encounter any errors in the below cell, please try to restart the kernel (Kernel -> Restart & Run All)</font>

In [ ]:
req_packages = [    
                    {"hbp_service_client"       : {"min_version": "1.1.0",  "install_version": "1.1.0"}},
                    {"sciunit"                  : {"min_version": "0.2.1",  "install_version": "0.2.1"}},
                    {"morphounit"               : {"min_version": "1.0.2",  "install_version": "1.0.2"}},
                    {"hbp_validation_framework" : {"min_version": "", "install_version": "https://github.com/appukuttan-shailesh/hbp-validation-client@usecase_patching"}},
                    {"neurom"                   : {"min_version": "1.4.10", "install_version": "1.4.10"}},
                    {"numpy"                    : {"min_version": "1.16.2", "install_version": "1.16.2"}},    
                    {"fpdf"                     : {"min_version": "1.7.2",  "install_version": "1.7.2"}},
                    {"PyPDF2"                   : {"min_version": "1.26.0", "install_version": "1.26.0"}},
                    {"tornado"                  : {"min_version": "4.5.3",  "install_version": "4.5.3"}}
                ]

def install_req_packages():
    # currently handles installations via PyPI and GitHub
    for pkg in req_packages:        
        for pkg_name, pkg_vinfo in pkg.items():
            print("Checking for package: {}".format(pkg_name))        
            try:
                pkg_resources.get_distribution(pkg_name)        
                current_version = parse_version(pkg_resources.get_distribution(pkg_name).version)
                print("\t{}: current version = {}".format(pkg_name, current_version))
                if not pkg_vinfo["min_version"] or current_version < parse_version(pkg_vinfo["min_version"]) or current_version > parse_version(pkg_vinfo["install_version"]):                                                
                        print("\tInstalling another version of {}.".format(pkg_name))
                        raise
            except:            
                if "github.com" in pkg_vinfo["install_version"]:
                    os.system("pip install --quiet --no-cache-dir --force-reinstall git+{}".format(pkg_vinfo["install_version"]))
                else:
                    os.system("pip install --quiet --no-cache-dir --force-reinstall {}=={}".format(pkg_name, pkg_vinfo["install_version"]))                                
                print("\t{}: installed version = {}".format(pkg_name, pkg_vinfo["install_version"]))
                if pkg_name == "hbp_service_client":
                    from IPython.display import HTML
                    display(HTML('''<script>window.requestAnimationFrame(() => { Jupyter.notebook.kernel.restart(); \
                    Jupyter.notebook.dirty = false; window.location.reload(); })</script>'''))

install_req_packages()                 

In [ ]:
import json
import urllib
import sciunit
import shutil
import pandas as pd
from pandas.io.json import json_normalize
from morphounit.utils import neuroM_loader, NeuroM_MorphStats 
from hbp_validation_framework import utils, TestLibrary, ModelCatalog

### Check if Model Catalog and Validation Framework Apps Exist in Collab
If the notebook is run inside a Collab, we check if an instance of the Model Catalog and Validation Framework apps exist in the current Collab. If not, we add an instance of each (this will be reflected in the Collab's navigation panel, possibly on reloading the page).

NOTE: **HBP_USERNAME** is an optional parameter when the notebook is being run inside the Collaboratory. The notebook can automatically identify your username in this scenario. This parameter needs to be specified if a user wishes to download the notebook and run it locally. Another potential (less likely) reason for specifying this (even within the Collaboratory) is in dealing with access permissions (wanting to run the test with different credentials).

NOTE: Even if this notebook is not run inside a Collab, the following cell needs to be executed. It will identify if environment and manage accordingly. When not run inside a Collab, it will simply setup parameters required for the test, and not attempt to create new apps.

In [ ]:
# your HBP username; not essential if running inside the Collaboratory
HBP_USERNAME = ""
testLibrary = TestLibrary(username=HBP_USERNAME, environment="integration")
modelCatalog = ModelCatalog.from_existing(testLibrary)

try:
    collab_path = get_collab_storage_path()
    collab_id = collab_path[1:] # this might fail for very old Collabs which use name instead of Collab ID
except:
    # not run inside Collaboratory
    print("\nPlease enter a Collab ID where you wish to store the results:")
    print("E.g.: 8123")
    print("Note: you should be a member of this Collab!")
    collab_id = input()
    if not isinstance(collab_id, int):
        raise ValueError("Possibly invalid Collab ID: {}. Numeric input expected!".format(collab_id))    

# check if apps exist; if not then create them
MCapp_navID = modelCatalog.exists_in_collab_else_create(collab_id)
modelCatalog.set_app_config(collab_id=collab_id, app_id=MCapp_navID, only_if_new="True")
VFapp_navID = testLibrary.exists_in_collab_else_create(collab_id)
testLibrary.set_app_config(collab_id=collab_id, app_id=VFapp_navID, only_if_new="True")

### Model Selection: Specifying model from ModelCatalog
The user is given an option to choose from existing compatible models.

In [ ]:
m1 = modelCatalog.get_model(model_id="f72f9e36-bb19-496a-8260-7090760319de") # 2e196088-b39e-4975-a3bb-6dd6d52ddb3f
m2 = modelCatalog.get_model(model_id="53934bfe-2400-4ba5-852e-be4628d1998e") # fee9a5eb-fbf1-4bb8-998c-992e3408fd3c
m3 = modelCatalog.get_model(model_id="e5b7b6dd-753c-4e09-999a-9c4e1c2e5177") # 25f0ab7b-33a3-48db-8375-7e31dc385c22
list_of_models = [m1, m2, m3]
df = pd.DataFrame.from_dict(json_normalize(list_of_models), orient='columns')
df = df.reindex(['name', 'id', 'author', 'brain_region', 'species', 'cell_type', 'model_scope', 'abstraction_level', 'description'], axis=1)
df.index += 1 
print("Available models are listed below:")
df.replace('\n','', regex=True)

In [ ]:
print("Enter the # of required model: ")
choice = input()
if choice <= len(list_of_models) and choice > 0:   
    model_id = list_of_models[choice-1]["id"]
    model_name = list_of_models[choice-1]["name"]
else:
    raise ValueError("Invalid entry for model choice!")

### Gather Additional Info
Currently the test is applicable for three cell types: _D1-type MSN_, _D2-type MSN_ and _FS Interneurons_ <br />
The usecase tries to identify the cell type from the input directory. If this isn't possible, the user is prompted to specify the cell type.

In [ ]:
# will try to determine cell type from model name; if not then ask user
# current options: msn_d1, msn_d2, fs
cell_type_map = {"msn_d1" : "medium spiny neuron (D1 type)",
                 "msn_d2" : "medium spiny neuron (D2 type)",
                 "fs"     : "fast spiking interneuron"}

if "msn_d1" in model_name:
    cell_type = "msn_d1" 
elif "msn_d2" in model_name:
    cell_type = "msn_d2"
elif "fs" in model_name:
    cell_type = "fs"    
else:
    print("\nPlease enter the cell_type: ")
    options = ["msn_d1", "msn_d2", "fs"]
    for i, each in enumerate(options,start=1):
        print("\t{}. {}".format(i,each))
    print("Enter the # of cell_type: ")
    choice = input()
    cell_type = options[choice-1]
print("Cell Type = {}".format(cell_type))

### Selection of desired model instance
The model has several model instances corresponding to various morphologies. These are listed below. The user needs to select one of these model instances.

In [ ]:
model_instances = modelCatalog.list_model_instances(model_id=model_id)
df = pd.DataFrame.from_dict(json_normalize(model_instances), orient='columns')
df = df.reindex(['version', 'id'], axis=1)
df.index += 1
pd.set_option('display.max_colwidth', -1)
df

The user is asked to specify the model instances that they are interested in validating. The valid range of values are indicated. The user can provide input as follows: <br />
 * All parameters, specify: `all` <br />
 * Single parameter set, specify number, e.g.: `1` <br />
 * Multiple parameter sets, specify numbers as a list, e.g.: `[1,4,5,8]` <br />

In [ ]:
print("Enter the model instance(s) to be validated: 1 - {}".format(len(model_instances)))
print("Example inputs: 1, 4, [1,4,5,8], all")
instances_entry = raw_input().lower()
if instances_entry == "all":
    instances_list = range(1, len(model_instances)+1)
else:    
    if isinstance(eval(instances_entry), list):
        instances_list = eval(instances_entry)        
    elif isinstance(eval(instances_entry), int):
        instances_list = [eval(instances_entry)]
    else:
        raise ValueError("Invalid entry for parameter set!")  
       
valid_instances_list = []
for i in instances_list:
    if i > 0 and i <= len(model_instances):
        valid_instances_list.append(i)
    else:
        print("Invalid entry: {}. Excluded.".format(i))

## Part 1: Hard Constraints Validation

### Instantiating the model; Running the validation tests
Validations for hard constraints are run for each of the selected morphologies. At the end of the test, the user is provided with a textual summary of the _score_ and the path to related output files generated by the test. These and other details can be viewed in the  _Validation Framework_ app (see Collab's Navigation panel; select `Validation Framework`).

In [ ]:
result_uuids = []

for index in valid_instances_list:
    model_instance_name = model_instances[index-1]["version"]    
    model_source = model_instances[index-1]["source"]
    model_path = os.path.abspath(urllib.urlretrieve(model_source, os.path.basename(model_source))[0])
    morph_model = neuroM_loader(model_path=model_path, name=model_instance_name)
    morph_model.model_uuid = model_id
    morph_model.model_version = model_instances[index-1]["version"]
       
    if cell_type in ["msn_d1", "msn_d2"]:
        test_alias = "basalg_msn_morph_hardChecks"
    else:
        test_alias = "basalg_fs_morph_hardChecks"
    result_id, score = utils.run_test(username=HBP_USERNAME, environment="integration", model=morph_model, test_alias=test_alias, test_version="1.0", storage_collab_id=collab_id, register_result=True, client_obj=testLibrary)
    result_uuids.append(result_id)    

print("The result(s) can be viewed in the HBP Validation Framework app. Direct link(s):")
for result_uuid in result_uuids:
    print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}?state=result.{}".format(str(collab_id),str(VFapp_navID), result_uuid))

### Score Summary

In [ ]:
if len(result_uuids) > 0:
    df, excluded_results = utils.generate_score_matrix(environment="integration", result_list=result_uuids, collab_id=collab_id, client_obj=modelCatalog)        
    from IPython.core.display import HTML
    HTML("<style>.rendered_html th {max-width: 120px;}</style>")
    display(df)

### Generate Report
The validation framework can generate a PDF report for all model variants (HoF parameter sets employed) that successfully completed the test. The user is prompted whether such a report should be generated for the current tests. If asked to generate, the location to the generated PDF is indicated.

In [ ]:
report_path = None
if len(result_uuids) > 0:
    print("\nDo you wish to generate a report of the tests executed?")
    print("Enter: y/n")
    choice = raw_input().lower()
    valid_choices = {"yes": True, "y": True, "no": False, "n": False}
    if valid_choices[choice]:
        valid_uuids, report_path = utils.generate_report(environment="integration", result_list=result_uuids, only_combined=True, client_obj=modelCatalog)    

### View Report Inside Jupyter Notebook
The PDF report created in the above cell is displayed within the Jupyter notebook. This can also be downloaded by clicking the download button inside the display frame.

In [ ]:
print report_path
if report_path:
    rel_report_path = os.path.relpath(report_path)
    from IPython.display import IFrame    
    display(IFrame(rel_report_path, width=900, height=650))

## Part 2: Soft Constraints Validation
[Currently only available for Fast Spiking Interneurons]

In [ ]:
if cell_type != "fs":
    print("Soft constraints validation not currently available for cell type = {}".format(cell_type))
    result_uuids = []

### Instantiating the model; Running the validation tests
Validations for soft constraints are run for only the morphologies that _pass_ the first part of the test. At the end of the test, the user is provided with a textual summary of the _mean score_ value (across the features tested) and the path to related output files generated by the test. These and other details can be viewed in the  _Validation Framework_ app (see Collab's Navigation panel; select `Validation Framework`).

In [ ]:
# Obtain instance_uuid of all above morphologies that passed the hard constraints validations
# Only these will be evaluated for the soft constraints
passed_morph_inst_uuid = []
for uuid in result_uuids:
    result = testLibrary.get_result(result_id=uuid)
    if result["results"][0]["passed"]:
        passed_morph_inst_uuid.append(result["results"][0]["model_version_id"])              

models_path = '/tmp/neuroM_morph_softChecks'
if os.path.exists(models_path):
    shutil.rmtree(models_path)
os.makedirs(models_path)

In [ ]:
import morphounit.tests

# Cycling through all above morphologies that passed the hard constraints validations
test_alias = 'morph_stats_Test'

neuroM_stats_file = 'fs_cells_NeuroM_MorphStats_pred.json'

result_uuids = []

file_entries = os.listdir(models_path)
morph_files = [f for f in file_entries if f.endswith(".swc")]

for uuid in passed_morph_inst_uuid:
    model_instance = modelCatalog.get_model_instance(instance_id=uuid)
    model_name = model_instance["version"]
    model_source = model_instance["source"]
    morph_path = os.path.abspath(urllib.urlretrieve(model_source, os.path.join(models_path, os.path.basename(model_source)))[0])

    morph_model = NeuroM_MorphStats(model_name=model_name, morph_path=morph_path,
                                    neuroM_pred_file=neuroM_stats_file)
    
    print morph_path, "---", model_name, '\n'
    
    morph_model.model_uuid = model_id          # uuid for the whole cell collection
    morph_model.model_version = model_name     # basename of each morphology file

    result_id, score = utils.run_test(username=HBP_USERNAME, environment="integration", 
                               model=morph_model, test_alias=test_alias, test_version="2.0",
                               storage_collab_id=collab_id, register_result=True, 
                               client_obj=testLibrary)

    result_uuids.append(result_id)

if len(result_uuids) > 0:   
    print("The result(s) can be viewed in the HBP Validation Framework app. Direct link(s):")
for result_uuid in result_uuids:
    print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}?state=result.{}".format(str(collab_id),str(VFapp_navID), result_uuid))       

### Score Summary

In [ ]:
if len(result_uuids) > 0:
    df, excluded_results = utils.generate_score_matrix(environment="integration", result_list=result_uuids, collab_id=collab_id, client_obj=modelCatalog)        
    from IPython.core.display import HTML
    HTML("<style>.rendered_html th {max-width: 120px;}</style>")
    display(df)

### Generate Report
The validation framework can generate a PDF report for all model variants that successfully completed the test. The user is prompted whether such a report should be generated for the current tests. If asked to generate, the location to the generated PDF is indicated.

In [ ]:
report_path = None
if len(result_uuids) > 0:
    print("\nDo you wish to generate a report of the tests executed?")
    print("Enter: y/n")
    choice = raw_input().lower()
    valid_choices = {"yes": True, "y": True, "no": False, "n": False}
    if valid_choices[choice]:
        valid_uuids, report_path = utils.generate_report(environment="integration", result_list=result_uuids, only_combined=True, client_obj=modelCatalog)    

### View Report Inside Jupyter Notebook
The PDF report created in the above cell is displayed within the Jupyter notebook. This can also be downloaded by clicking the download button inside the display frame.

In [ ]:
if report_path:
    rel_report_path = os.path.relpath(report_path)
    from IPython.display import IFrame    
    display(IFrame(rel_report_path, width=900, height=650))